In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, datetime
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [2]:

# 设置随机种子以确保可复现性
random.seed(42)
np.random.seed(42)

# 创建数据生成的基本参数
num_rows = 100000
person_id_range = (1, 30000)  # 假设有3000个不同的患者
disease_list = ['A', 'B', 'C', 'D','E','F']  # 假设疾病列表
date_range = (datetime(2000, 1, 1), datetime(2023, 12, 31))  # 日期范围从2019年到2023年

# 生成随机数据
data = {
    'person_id': np.random.randint(*person_id_range, size=num_rows),
    'event_concept_name': np.random.choice(disease_list, size=num_rows),
    'svcdate': [date_range[0] + timedelta(days=random.randint(0, (date_range[1] - date_range[0]).days)) for _ in range(num_rows)]
}

# 创建DataFrame
df = pd.DataFrame(data)

print(df.head())  # 打印前几行以查看数据


   person_id event_concept_name    svcdate
0      23655                  B 2004-12-29
1      15796                  A 2001-02-13
2        861                  F 2012-05-03
3       5391                  B 2010-12-26
4      29803                  D 2010-01-05


In [3]:

# 设置随机种子以确保可复现性
random.seed(41)
np.random.seed(41)

# 创建数据生成的基本参数
num_rows = 1000000
person_id_range = (1, 300000)  # 假设有3000个不同的患者
disease_list = ['A', 'B', 'C', 'D','E','F']  # 假设疾病列表
date_range = (datetime(2000, 1, 1), datetime(2023, 12, 31))  # 日期范围从2019年到2023年

# 生成随机数据
data2 = {
    'person_id': np.random.randint(*person_id_range, size=num_rows),
    'event_concept_name': np.random.choice(disease_list, size=num_rows),
    'svcdate': [date_range[0] + timedelta(days=random.randint(0, (date_range[1] - date_range[0]).days)) for _ in range(num_rows)]
}

# 创建DataFrame
df2 = pd.DataFrame(data2)

print(df2.head())  # 打印前几行以查看数据


   person_id event_concept_name    svcdate
0     296897                  D 2017-02-02
1     132004                  C 2014-11-26
2     217059                  C 2010-05-08
3     250961                  E 2007-06-13
4      53492                  D 2017-04-19


In [4]:
class DiseaseCount:
    """
    Creata a class and load original data frame to class and make data processing to count patients in each year
    """
    def __init__(self, name):
        """
        Initiate class with None attributes

        Args:
            name (str): give a name to the class
        """
        self.original_df = None # Initiate the original data frame with none
        self.unique_id_df = None # Initiate the unique patients id data frame 
        self.date_order_df = None # Initiate the date order dataframe
        self.year_counts_df = None # Initiate the year count dataframe
        self.lag_counts_df = None # Initiate the lag count dataframe
        self.lag_num = None # Initiate the lag num
        self.name = name # Give the argument name to the class
        self.new_year_data = None # Initiate the new year data dataframe
    
    def load_data(self, data):
        """
        load the data to the object as the original table aka df
        
        args:
        - data(dataframe or other format table): the data which will be added into the object as the
                original table aka df, could be a dataframe, also could be other format and transfer
                to dataframe
        """
        
        if isinstance(data, pd.DataFrame):
            self.original_df = data
        else:
            self.original_df = pd.DataFrame(data)
            
    def unique_patient(self):
        """
        Only keep the unique patient ID 
        """
        
        if self.original_df is None:
            raise ValueError("Data is not loaded. Please load data first using `load_data` method.")

        # sort by person id and svcdate
        sorted_df = self.original_df.sort_values(by=['person_id', 'svcdate'])

        # get the first condition recording and generate a copy
        first_records = sorted_df.drop_duplicates(subset=['person_id'], keep='first').copy()
        
        self.unique_id_df = first_records
        
    def svcdate_order(self):
        """
        Prepare for count function to reorder the dataframe as svc_data order
        """
        
        if self.unique_id_df is None:
            raise ValueError("Data is not loaded. Please load data first using `unique_patient` method.")
        
        
        sorted_df = self.unique_id_df.sort_values(by = ['svcdate'])
        
        self.date_order_df = sorted_df
        
    def year_counts(self):
        """
        Make yearly counts by the years
        """
        if self.date_order_df is None:
            raise ValueError("Data is not loaded. Please load data first using `svcdate_order` method.")
    
        # Extract the year from the 'svcdate' column
        self.date_order_df['year'] = self.date_order_df['svcdate'].dt.year
        
        # Count the unique patients per year
        year_counts_df = self.date_order_df.groupby('year').agg(patient_count=('person_id', 'nunique')).reset_index()
        
        self.year_counts_df = year_counts_df
        
    def generate_lag(self, lag = 3):
        """
        Generate the dataframe with showing lag years based on the 'year_counts_df'.
        
        args:
        - lag(int type): The number of lag years to calculate.
        """    
        temp_df = self.year_counts_df.copy()
        
        year_to_count = self.year_counts_df.set_index('year')['patient_count'].to_dict()
        
        for i in range(1, lag + 1):
            temp_df[f'lag_{i}'] = temp_df['year'].apply(lambda y: year_to_count.get(y - i, 0))
            
        temp_df = temp_df.iloc[lag:].reset_index(drop=True)    
        
        self.lag_num = lag
        self.lag_counts_df = temp_df
        
    def generate_next_yearlag(self):
        """
        Generate the next year lag prepare for prediction
        """
        # Get the maximum year in the current lag_counts_df
        last_year = self.lag_counts_df['year'].max()
        
        # Create a dictionary to hold the new row data
        new_year_data = {'year': last_year + 1}
        
        # Calculate lag values for the next year based on existing data
        year_to_count = self.year_counts_df.set_index('year')['patient_count'].to_dict()
        for i in range(1, self.lag_num + 1):
            new_year_data[f'lag_{i}'] = year_to_count.get(last_year + 1 - i, 0)
        self.new_year_data = pd.DataFrame([new_year_data]) 
        # Append this new row to lag_counts_df
        # self.lag_counts_df = self.lag_counts_df.append(new_year_data, ignore_index=True)
            
            
        

In [5]:
t1 = DiseaseCount("table1")
t2 = DiseaseCount("table2")

t1.load_data(data)
t2.load_data(data2)

t1.unique_patient()
t2.unique_patient()

t1.svcdate_order()
t2.svcdate_order()

t1.year_counts()
t2.year_counts()

t1.generate_lag()
t2.generate_lag()

t1.generate_next_yearlag()
t2.generate_next_yearlag()
t2.lag_counts_df

# t1.year_counts_df
# t2.year_counts_df

,year,patient_count,lag_1,lag_2,lag_3
0,2003,25480,29378,33684,39029
1,2004,22269,25480,29378,33684
2,2005,19321,22269,25480,29378
3,2006,17016,19321,22269,25480
4,2007,14666,17016,19321,22269
5,2008,13117,14666,17016,19321
6,2009,11255,13117,14666,17016
7,2010,9782,11255,13117,14666
8,2011,8348,9782,11255,13117
9,2012,7269,8348,9782,11255


In [6]:
t2.new_year_data

,year,lag_1,lag_2,lag_3
0,2024,1563,1772,2164


In [34]:
class LinearRegModel:
    def __init__(self, name):
        """
        Initiate the Linear regression model 
        Args:
            name ( str type ): give this model a name
        """
        self.name = name # Give this linear regression model name
        self.indep_df = None # initiate the independent variable table which is the X
        self.dep_df = None # initiate the dependent variable table which is the Y
        self.indep_nextyear_df = None # initiate the independent variable table which is the X
        self.dep_nextyear_df = None # initiate the dependent variable table which is the Y
        self.indep_name = None # initiate the independent name as None
        self.dep_name = None # initiate the dependent name as None
        self.models = None # initiate the regression model
        self.combined_df = None # initiate the combined dataframe
        self.combined_predictor = None # initiate the predictor param dataframe
        self.lag_num = None # Initiate the lag number 
        self.prediction = None # Initiate the prediction part
        self.lag_covariance = None # Initiate the lag covariance 
        
        
    def load_data(self, dep, indep):
        """
        load the independent table and dependent table into this class
        
        args:
            dep( object ): should be a object which is the dependent variable
            indep( object ): should be a object which is the independent variable
        """
        
        if dep.lag_counts_df is None or indep.lag_counts_df is None:
            raise ValueError("Data is not loaded. Please load data to ge` method.")
        
        self.indep_df = indep.lag_counts_df
        self.dep_df = dep.lag_counts_df
        self.indep_nextyear_df = indep.new_year_data
        self.dep_nextyear_df = dep.new_year_data
        self.indep_name = indep.name
        self.dep_name = dep.name
        self.lag_num = dep.lag_num
        
    def combine_data(self):
        """ 
        Generate combination data with the two dataframe
        """
        
        if  self.indep_df is None or self.dep_df is None:
            raise ValueError("Data is not loaded. Please use load_data function firstly.")
        
        if 'year' not in self.indep_df.columns or 'year' not in self.dep_df.columns:
            raise ValueError("Both data tables must have a 'year' column to combine based on it.")
            
        self.combined_df = pd.merge(self.dep_df, self.indep_df,  on='year', suffixes=( '_' + self.dep_name, '_' + self.indep_name))
        start_year = self.combined_df['year'].min()
        self.combined_df['T'] = self.combined_df['year'] - start_year + 1
        self.combined_predictor = pd.merge(self.dep_nextyear_df, self.indep_nextyear_df,  on='year', suffixes=( '_' + self.dep_name, '_' + self.indep_name))
        # self.combined_predictor.drop(['year'])
        self.combined_predictor['T'] = self.combined_df['T'].iloc[-1] + 1
        
    def get_covarience(self):
        """
        Calculate the covariance between table1 and table2 and add it to the combined dataframe.
        """
        if self.combined_df is None:
            raise ValueError("Data is not combined. Please use combine_data function first.")
        
        overall_covariance = np.cov(self.combined_df['patient_count_table1'], self.combined_df['patient_count_table2'])[0, 1]
        
        self.combined_df['overall_covariance'] = overall_covariance
        self.combined_predictor['overall_covariance'] = overall_covariance
    
    def calculate_lag_covariance(self):
        """
        Calculate the lags covariance between table1 and table2 and add it to the combined dataframe.
        """
        # if self.combined_df is None:
        #     raise ValueError("Data is not combined. Please use combine_data function first.")
        
        # table1_lags = np.array([self.combined_df[f'lag_{i}_'+self.dep_name] for i in range(1, self.lag_num+1)])
        # table2_lags = np.array([self.combined_df[f'lag_{i}_'+self.indep_name] for i in range(1, self.lag_num+1)])
        # covariance_matrix = np.cov(table1_lags, table2_lags)
        # self.lag_covariance = covariance_matrix[0, 1]
        # self.combined_df['lags_covariance'] = self.lag_covariance
        
        # if self.combined_predictor is None:
        #     raise ValueError("Data is not predictor combined. Please use combine_data function first.")
        
        # table1_lags_pred = np.array([self.combined_predictor[f'lag_{i}_'+self.dep_name] for i in range(1, self.lag_num+1)])
        # table2_lags_pred = np.array([self.combined_predictor[f'lag_{i}_'+self.indep_name] for i in range(1, self.lag_num+1)])
        # covariance_matrix_pred = np.cov(table1_lags_pred, table2_lags_pred)
        # self.lag_covariance_pred = covariance_matrix_pred[0, 1]
        # self.combined_predictor['lags_covariance'] = self.lag_covariance_pred
        
        def calculate_covariance(row, num_lags):
            table1_lags = np.array([row[f'lag_{i}_table1'] for i in range(1, num_lags+1)])
            table2_lags = np.array([row[f'lag_{i}_table2'] for i in range(1, num_lags+1)])
            covariance_matrix = np.cov(table1_lags, table2_lags)
            return covariance_matrix[0, 1]

            # Compute the covariance for each row and store it in a new column
        self.lag_covariance = self.combined_df.apply(calculate_covariance, num_lags= 3, axis=1)
        self.combined_df['lags_variance'] = self.lag_covariance
        
        self.lag_covariance_pred = self.combined_predictor.apply(calculate_covariance, num_lags= 3, axis=1)
        self.combined_predictor['lags_variance'] = self.lag_covariance_pred
        
    def fit_model(self):
        """
        Fit the linear regression model using statsmodels and display the summary.
        """
        if self.combined_df is None:
            raise ValueError("Data is not combined. Please use combine_data function first.")

        # Assuming the dependent variable column is from dep_df and others are independent variables
        X = self.combined_df.drop(['year', 'patient_count_' + self.dep_name, 'patient_count_' + self.indep_name], axis=1)
        y = self.combined_df['patient_count_' + self.dep_name]
        
        # Add a constant to the model (for the intercept)
        X = sm.add_constant(X)
        
        # Fit the model
        model = sm.OLS(y, X).fit()
        
        # Store the model
        self.models = model
        
        # Print the summary
        # print(model.summary()) 
        
    def generate_prediction(self):
        """
        Predict the dependent variable next year patient count based on the linear regression model.
        """
        if self.models is None:
            raise ValueError("No model is fitted. Please use fit_model function first.")
        
        
        # Add a constant to the model (for the intercept)
        next_year_data = self.combined_predictor.drop(['year'], axis=1).copy()
        
        next_year_data = sm.add_constant(next_year_data)
        # print(next_year_data)
        # Predict using the fitted model
        prediction = self.models.predict(next_year_data)
        # predicted_values = prediction[:, 0]
        self.prediction = prediction
        print("The forecast value for next year new patients count is " + str(prediction.iloc[0]))
        
        

In [38]:
lr = LinearRegModel('l1')
lr.load_data(t1, t2)
lr.combine_data()
lr.get_covarience()
lr.calculate_lag_covariance()
lr.fit_model()
lr.generate_prediction()
lr.combined_df
lr.models.summary()
# lr.combined_predictor

The forecast value for next year new patients count is 147.80572172505182


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     patient_count_table1   R-squared:                       0.999
Model:                              OLS   Adj. R-squared:                  0.999
Method:                   Least Squares   F-statistic:                     1860.
Date:                  Wed, 15 May 2024   Prob (F-statistic):           2.29e-17
Time:                          11:50:31   Log-Likelihood:                -92.637
No. Observations:                    21   AIC:                             203.3
Df Residuals:                        12   BIC:                             212.7
Df Model:                             8                                         
Covariance Type:              nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
lag_1_table1          -0.8256      0.289     -2.859      0.014      -1.455      -0.196
lag_2_table1          -0.5950      0.284     -2.098      0.058      -1.213       0.023
lag_3_table1          -0.3488      0.267     -1.309      0.215      -0.930       0.232
lag_1_table2           0.1193      0.083      1.444      0.174      -0.061       0.299
lag_2_table2           0.1205      0.066      1.832      0.092      -0.023       0.264
lag_3_table2           0.0148      0.065      0.226      0.825      -0.127       0.157
T                      4.1096      8.101      0.507      0.621     -13.540      21.759
overall_covariance -1.067e-05   3.48e-05     -0.307      0.764   -8.65e-05    6.52e-05
lags_variance          0.0002      0.000      1.371      0.195   -9.87e-05       0.000
==============================================================================
Omnibus:                        1.094   Durbin-Watson:                   2.484
Prob(Omnibus):                  0.579   Jarque-Bera (JB):                0.154
Skew:                           0.046   Prob(JB):                        0.926
Kurtosis:                       3.409   Cond. No.                     7.30e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.3e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df
data = {
    'year': range(2003, 2024),
    'patient_count_table1': [2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247, 193, 210, 155],
    'lag_1_table1': [3005, 2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247, 193, 210],
    'lag_2_table1': [3397, 3005, 2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247, 193],
    'lag_3_table1': [3885, 3397, 3005, 2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247],
    'patient_count_table2': [25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380, 2164, 1772, 1563],
    'lag_1_table2': [29378, 25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380, 2164, 1772],
    'lag_2_table2': [33684, 29378, 25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380, 2164],
    'lag_3_table2': [39029, 33684, 29378, 25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380]
}
df = pd.DataFrame(data)

# Function to calculate the covariance between the lagged values of two tables
def calculate_covariance(row, num_lags):
    table1_lags = np.array([row[f'lag_{i}_table1'] for i in range(1, num_lags+1)])
    table2_lags = np.array([row[f'lag_{i}_table2'] for i in range(1, num_lags+1)])
    covariance_matrix = np.cov(table1_lags, table2_lags)
    return covariance_matrix[0, 1]

# Compute the covariance for each row and store it in a new column
df['covariance'] = df.apply(calculate_covariance, num_lags= 3, axis=1)

print(df[['year', 'covariance']])



    year    covariance
0   2003  2.131532e+06
1   2004  1.686647e+06
2   2005  1.347950e+06
3   2006  1.050882e+06
4   2007  7.249912e+05
5   2008  5.414925e+05
6   2009  3.704825e+05
7   2010  2.827878e+05
8   2011  3.520552e+05
9   2012  1.675815e+05
10  2013  1.238298e+05
11  2014  1.326330e+05
12  2015  5.820100e+04
13  2016  3.746783e+04
14  2017  5.916900e+04
15  2018  3.809100e+04
16  2019  2.128933e+04
17  2020  1.877017e+04
18  2021  1.543783e+04
19  2022  1.402600e+04
20  2023  4.582667e+03


In [10]:
import pandas as pd
import numpy as np

# 示例数据
data = {
    'year': [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'patient_count_table1': [2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247, 193, 210, 155],
    'lag_1_table1': [3005, 2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247, 193, 210],
    'lag_2_table1': [3397, 3005, 2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247, 193],
    'lag_3_table1': [3885, 3397, 3005, 2574, 2250, 1891, 1705, 1426, 1338, 1099, 918, 869, 716, 590, 573, 494, 406, 369, 324, 286, 247],
    'patient_count_table2': [25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380, 2164, 1772, 1563],
    'lag_1_table2': [29378, 25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380, 2164, 1772],
    'lag_2_table2': [33684, 29378, 25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380, 2164],
    'lag_3_table2': [39029, 33684, 29378, 25480, 22269, 19321, 17016, 14666, 13117, 11255, 9782, 8348, 7269, 6455, 5641, 4882, 4222, 3679, 3182, 2777, 2380],
    'T': list(range(1, 22))
}

# 转换为 DataFrame
df = pd.DataFrame(data)

# 计算总协方差并添加到 DataFrame
covariances = []
for index, row in df.iterrows():
    # 提取当前行的 table1 和 table2 的滞后值
    lags_table1 = [row['lag_1_table1'], row['lag_2_table1'], row['lag_3_table1']]
    lags_table2 = [row['lag_1_table2'], row['lag_2_table2'], row['lag_3_table2']]
    
    # 计算两组滞后值的协方差矩阵，并获取协方差值
    covariance_value = np.cov(lags_table1, lags_table2)[0, 1]
    covariances.append(covariance_value)

# 添加协方差列到 DataFrame
df['overall_covariance'] = covariances

# 打印结果
print(df[['year', 'overall_covariance']])



    year  overall_covariance
0   2003        2.131532e+06
1   2004        1.686647e+06
2   2005        1.347950e+06
3   2006        1.050882e+06
4   2007        7.249912e+05
5   2008        5.414925e+05
6   2009        3.704825e+05
7   2010        2.827878e+05
8   2011        3.520552e+05
9   2012        1.675815e+05
10  2013        1.238298e+05
11  2014        1.326330e+05
12  2015        5.820100e+04
13  2016        3.746783e+04
14  2017        5.916900e+04
15  2018        3.809100e+04
16  2019        2.128933e+04
17  2020        1.877017e+04
18  2021        1.543783e+04
19  2022        1.402600e+04
20  2023        4.582667e+03
